# Изучение множеств графов с одинаковыми сигнатурами.

Под сигнатурой графа подразумевается набор $(d_1, d_2 , \ldots d_n)$, где $d_i$ --- степень i-ой вершины неориентированного графа.

## Часть 1. Реализация функций (можно пропустить)

In [3]:
import networkx as nx
import matplotlib.pylab as plt
import numpy as np
import itertools
import copy
% matplotlib inline

In [4]:
def get_graph_signature(g):
    signature = []
    for node in list(g.nodes()):
        signature.append(g.degree(node))
    return tuple(signature)

def get_all_edges(n_nodes):
    s = np.arange(1, n_nodes + 1)
    return list(itertools.combinations(s, 2))

def get_graphs_dict(n_nodes, edges_min=0, edges_max=None, max_edges=False):
    if max_edges == True:
        edges_max = n_nodes * (n_nodes - 1) // 2
    elif edges_max == None:
        edges_max = edges_min + 1
    edges = get_all_edges(n_nodes)
    graphs_dict = {}
    for num_edges in range(edges_min, edges_max + 1):
        edges_combinations = list(itertools.combinations(edges, num_edges))
        for edg in edges_combinations:
            g = nx.Graph()
            g.add_edges_from(edg)
            sign = tuple(dict(g.degree()).values())
            if sign not in graphs_dict.keys():
                graphs_dict[sign] = [g]
            else:
                graphs_dict[sign].append(g)
    return graphs_dict

In [14]:
def plot_graphs_by_signature(signature, graphs_dict=None, ncols=7):
    n_nodes = len(signature)
    n_edges = np.sum(signature) // 2
    
    if graphs_dict == None:
        graphs_dict = get_graphs_dict(n_nodes, n_edges)
        assert (signature in graphs_dict.keys()), "No such signatures"
        
    graphs = graphs_dict[signature]
    
    if len(graphs) == 1: 
        pass
    elif (len(graphs) <= ncols):
        #plt.suptitle('Signature {}'.format(signature), fontsize=18)
        nrows=1
        ncols=len(graphs)
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(3*ncols, 3*nrows),)
        plt.suptitle('Signature {}\n\n'.format(signature), fontsize=18)
        
        for i, ax in enumerate(axes):
            ax.set_title('tau: {:.3f}'.format(estimate_tau(get_communication_matrix(graphs[i]), n_iter=1000 * n_nodes)))
            nx.draw(graphs[i], pos=nx.circular_layout(graphs[i]), ax=ax, with_labels=True, 
                            node_color='r', edge_color='black')
        plt.tight_layout()
        plt.subplots_adjust(top=0.80)
        plt.show()

        
    else:
        nrows = len(graphs) // ncols + 1
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(3*ncols, 3*nrows),)
        plt.suptitle('Signature {}\n\n'.format(signature), fontsize=18)
        
        for i in range(nrows):
            for j in range(ncols):
                if i * ncols + j < len(graphs):
                    gr = graphs[i*ncols + j]
                    
                    axes[i, j].set_title('tau: {:.3f}'.format(estimate_tau(get_communication_matrix(gr), n_iter=1000 * n_nodes)))
                    nx.draw(gr, pos=nx.circular_layout(gr), ax=axes[i, j], with_labels=True, 
                            node_color='r', edge_color='black')
                else:
                    axes[i,j].xaxis.set_visible(False)
                    axes[i,j].tick_params(left=False, labelleft=False)
                    axes[i,j].patch.set_visible(False)
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.85)
        plt.show()

In [6]:
def draw_graph(g):
    nx.draw(g, pos=nx.circular_layout(g), with_labels=True, 
                    node_color='r', edge_color='black')

In [7]:
def get_all_signatures(graphs_dict):
    keys = [key for key in graphs_dict.keys()]
    keys = list(set([tuple(sorted(key)) for key in keys]))
    keys = sorted(keys, key= lambda x: len(x))
    for i, key in enumerate(keys):
        print(i, key)
    return keys

In [8]:
def plot_several_signatures(signatures):
    n_nodes = len(signatures[0])
    edges = [sum(signature) // 2 for signature in signatures]
    graphs_dict = get_graphs_dict(n_nodes, np.min(edges), np.max(edges))
    
    for signature in signatures:
        print(signature, len(signatures))
        plot_graphs_by_signature(signature, graphs_dict)

In [9]:
def build_graph_from_adj(M):
    rows, cols = np.where(M - np.eye(M.shape[0]) > 0)
    rows += 1
    cols += 1
    edges = zip(rows, cols)
    g = nx.Graph()
    g.add_edges_from(edges)
    return g

In [10]:
def get_communication_matrix(graph):
    Lp = nx.to_numpy_matrix(graph)
    #print(Lp)
    N = Lp.shape[0]
    Lp = np.add(Lp, np.eye(N))
    #print(Lp)
    Lp = np.divide(Lp, np.sum(Lp, axis=1))
    #print(Lp)
    return Lp  

In [11]:
N = 5
E = []
for i in range(N):
    E_prime = []
    for j in range(N):
        M = np.eye(N)
        M[[i, j]] = M[[j, i]]
        E_prime.append(M)
    E.append(E_prime)
    
def swap_vertices(M, i, j):
    C = E[i][j]
    return np.dot(C.T, np.dot(M, C))

In [12]:
def estimate_tau(M, signature=None, n_iter=1000):
    taus = []
    N = M.shape[0]
    if signature == None:
        signature = np.sum(M > 0, axis=1)
        pi = np.array(signature)
    else: 
        pi = np.array(signature) + 1
    for _ in range(n_iter):
        pi = pi.reshape(N, 1)
        x = np.random.random(N).reshape(N, 1)
        ort = (np.dot(x.T, pi) /  np.dot(pi.T, pi)) * pi
        x -= ort
        x /= np.sqrt(np.dot(x.T, x))
        # print('Check:', np.dot(x, pi).round(2))
        Mx = np.dot(M, x)
        taus.append(np.dot(Mx.T, Mx))
    return np.max(taus)